In [101]:
import openai 
import pandas as pd
import re
from typing import Optional
from openai import OpenAI

df= pd.read_csv("tiktok.csv",delimiter=",")


api_key = ''#colocar tu key

client = openai.OpenAI(api_key=api_key)


df = df.dropna(axis=1, how='all')
df = df[["authorMeta/avatar", "authorMeta/name", "text", "diggCount", "shareCount", "playCount", "videoMeta/duration", 
         "musicMeta/musicName", "musicMeta/musicAuthor", "musicMeta/musicOriginal", "createTimeISO", "webVideoUrl"]]
df = df.rename(columns={
    "authorMeta/avatar": "authorMeta.avatar", 
    "authorMeta/name": "authorMeta.name",
    "videoMeta/duration": "videoMeta.duration",
    "musicMeta/musicName": "musicMeta.musicName",
    "musicMeta/musicAuthor": "musicMeta.musicAuthor",
    "musicMeta/musicOriginal": "musicMeta.musicOriginal"
})




In [102]:




def analyze_sentiment(text: Optional[str]) -> str:
    if not text:
        return "No text provided"
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", "content": "You are a helpful assistant that analyzes sentiment."},
                {"role": "user", "content": f"Analyze the sentiment of this text: '{text}', respond in Spanish, please"}
            ],
            max_tokens=100
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

print(analyze_sentiment(df.loc[0, 'text'])) 

El sentimiento del texto es positivo y refleja entusiasmo. La mención de ser estudiante foráneo sugiere una experiencia de aprendizaje y crecimiento personal, mientras que los emojis de casa y libros añaden un toque de calidez y estudio. Además, el uso de hashtags relacionados con la universidad y la vida estudiantil indica un sentido de comunidad y pertenencia. En general, la expresión transmite una vivencia alegre y significativa en el contexto académico.


In [103]:
df['sentiment_analysis'] = df['text'].apply(lambda x: analyze_sentiment(x))


In [104]:
df.head()

,authorMeta.avatar,authorMeta.name,text,diggCount,shareCount,playCount,videoMeta.duration,musicMeta.musicName,musicMeta.musicAuthor,musicMeta.musicOriginal,createTimeISO,webVideoUrl,sentiment_analysis
0,https://p16-sign-va.tiktokcdn.com/tos-maliva-a...,danyandrade_2,Un dia como estudiante Foráneo🏠📚 #vlog #foran...,12500,378,138000,77,A Midnight Jazz Affair,Dj Xolisi Khumalo,False,2024-09-21T02:14:38.000Z,https://www.tiktok.com/@danyandrade_2/video/74...,El sentimiento del texto es positivo. Se refle...
1,https://p16-sign-va.tiktokcdn.com/tos-maliva-a...,lanveryy2.0,#utec esto no es fun,326,22,13800,16,sonido original,trashed,True,2023-05-29T18:00:02.000Z,https://www.tiktok.com/@lanveryy2.0/video/7238...,El sentimiento de este texto es negativo. La e...
2,https://p16-sign-va.tiktokcdn.com/tos-maliva-a...,danyandrade_2,¿Me alcanzan $5.00? #vlog #utec #universidad #...,4313,93,53000,109,A Midnight Jazz Affair,Dj Xolisi Khumalo,False,2024-09-28T00:31:57.000Z,https://www.tiktok.com/@danyandrade_2/video/74...,El sentimiento del texto es neutral. La frase ...
3,https://p16-sign-sg.tiktokcdn.com/aweme/720x72...,uniquetradingcenter,My heart #gold #trader #livetrade #fypシ #trad...,772,304,13700,10,original sound - UTEC,UTEC,True,2024-08-31T08:13:31.000Z,https://www.tiktok.com/@uniquetradingcenter/vi...,"La sensación del texto parece ser positiva, ya..."
4,https://p16-sign-va.tiktokcdn.com/tos-maliva-a...,soyangelelcurioso,No más pobres en un país de Ricos 🤔 Sígueme en...,86700,2088,2900000,78,Love You So,The King Khan & BBQ Show,False,2022-09-01T12:00:06.000Z,https://www.tiktok.com/@soyangelelcurioso/vide...,El texto tiene una mezcla de sentimientos. La ...


In [105]:


def analyze_sentiment_percentages(text: Optional[str]) -> str:
    if not text:
        return "No text provided"
    
    api_key = ''#colocar key

    client = OpenAI(api_key=api_key)     
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that analyzes sentiment."},
                {"role": "user", "content": f"Analyze the following text and provide the percentage breakdown of the main emotions (happiness, sadness, anger, fear, surprise): '{text}', would you like to give it to me in list format (ex: ['happiness':0.25, ' sadness':0.25, 'anger':0.25, 'fear':0.25, 'surprise':0.25]), just return the list to me, nothing more "}
            ],
            max_tokens=100
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"



In [106]:
print(analyze_sentiment_percentages(df.loc[1, 'text']))  

['happiness': 0.0, 'sadness': 0.0, 'anger': 0.6, 'fear': 0.0, 'surprise': 0.4]


In [107]:
print(analyze_sentiment_percentages(df.loc[2, 'text'])) 

['happiness':0.10, 'sadness':0.10, 'anger':0.05, 'fear':0.05, 'surprise':0.70]


In [108]:
df['sentiment_analysis'] = df['text'].apply(lambda x: analyze_sentiment_percentages(x))


In [109]:
df.head()

,authorMeta.avatar,authorMeta.name,text,diggCount,shareCount,playCount,videoMeta.duration,musicMeta.musicName,musicMeta.musicAuthor,musicMeta.musicOriginal,createTimeISO,webVideoUrl,sentiment_analysis
0,https://p16-sign-va.tiktokcdn.com/tos-maliva-a...,danyandrade_2,Un dia como estudiante Foráneo🏠📚 #vlog #foran...,12500,378,138000,77,A Midnight Jazz Affair,Dj Xolisi Khumalo,False,2024-09-21T02:14:38.000Z,https://www.tiktok.com/@danyandrade_2/video/74...,"['happiness':0.60, 'sadness':0.05, 'anger':0.0..."
1,https://p16-sign-va.tiktokcdn.com/tos-maliva-a...,lanveryy2.0,#utec esto no es fun,326,22,13800,16,sonido original,trashed,True,2023-05-29T18:00:02.000Z,https://www.tiktok.com/@lanveryy2.0/video/7238...,"['happiness': 0.0, 'sadness': 0.2, 'anger': 0...."
2,https://p16-sign-va.tiktokcdn.com/tos-maliva-a...,danyandrade_2,¿Me alcanzan $5.00? #vlog #utec #universidad #...,4313,93,53000,109,A Midnight Jazz Affair,Dj Xolisi Khumalo,False,2024-09-28T00:31:57.000Z,https://www.tiktok.com/@danyandrade_2/video/74...,"['happiness': 0.10, 'sadness': 0.20, 'anger': ..."
3,https://p16-sign-sg.tiktokcdn.com/aweme/720x72...,uniquetradingcenter,My heart #gold #trader #livetrade #fypシ #trad...,772,304,13700,10,original sound - UTEC,UTEC,True,2024-08-31T08:13:31.000Z,https://www.tiktok.com/@uniquetradingcenter/vi...,"['happiness': 0.5, 'sadness': 0.0, 'anger': 0...."
4,https://p16-sign-va.tiktokcdn.com/tos-maliva-a...,soyangelelcurioso,No más pobres en un país de Ricos 🤔 Sígueme en...,86700,2088,2900000,78,Love You So,The King Khan & BBQ Show,False,2022-09-01T12:00:06.000Z,https://www.tiktok.com/@soyangelelcurioso/vide...,"['happiness':0.20, 'sadness':0.10, 'anger':0.1..."


In [110]:
df.loc[1, 'sentiment_analysis']

"['happiness': 0.0, 'sadness': 0.2, 'anger': 0.4, 'fear': 0.0, 'surprise': 0.4]"

In [111]:
def hashtags(text: Optional[str]) -> str:
    if not text:
        return "No text provided"
    
    api_key = '' # colocar key

    client = OpenAI(api_key=api_key)    
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", "content": "You are a helpful assistant that analyzes  the text and hashtags"},
                {"role": "user", "content": f"I want you to give me the hashtags used in this text:'{text}', I want you to give it to me in list format (e.g. ['#hello','#happy']), just return the list, nothing more"}
            ],
            max_tokens=100
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

In [112]:
print(hashtags(df.loc[2, 'text']))

['#vlog', '#utec', '#universidad', '#foraneo', '#elsalvador', '#diaenmivida', '#universidadtecnologica', '#campus', '#universidades', '#danielandrade', '#campus', '#sanvicente']


In [ ]:
df['hashtags'] = df['text'].apply(lambda x: hashtags(x))


In [94]:


def extract_hashtags(text):
    if isinstance(text, str):
        return re.findall(r'#\w+', text)
    return []

# Crear una nueva columna que cuente los hashtags en cada texto
df['hashtag_count'] = df['hashtags'].apply(lambda x: len(extract_hashtags(x)))

print(df.to_string())

                                                                                                                                                                                                                                                                  authorMeta.avatar      authorMeta.name                                                                                                                                                                                    text  diggCount  shareCount  playCount  videoMeta.duration     musicMeta.musicName     musicMeta.musicAuthor  musicMeta.musicOriginal             createTimeISO                                                            webVideoUrl                                                                   sentiment_analysis                                                                                                                                                                           hashtags  hashtag_count
0        https

In [114]:
def get_representative_words_from_column(df: pd.DataFrame, column_name: str) -> str:
    text = " ".join(df[column_name].astype(str).values)
    
    api_key = '' #colocar key

    client = OpenAI(api_key=api_key)        
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes text."},
                {"role": "user", "content": f"Analyze the following text and provide the 5 most representative words: '{text}'. Return only the list of 5 words, nothing more. I want you to return it to me this way (Ex: ['word1','word2','word3'])"}
            ],
            max_tokens=50
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

result = get_representative_words_from_column(df, 'text')
print(result)

['utec', 'universidad', 'estudiantes', 'comunicacion', 'elsalvador']
